In [3]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Flatten
from keras.applications.vgg16 import VGG16

In [4]:
from keras.applications import VGG16
conv_base = VGG16(weights='imagenet',
    include_top = False,
    input_shape=(224,224,3))

In [5]:
conv_base.trainable = True

set_trainable = False

for layer in conv_base.layers:
  #if layer.name == 'conv2d_81':
  if layer.name=='input_1':
    set_trainable = True
  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False


In [6]:
initializer = tensorflow.keras.initializers.HeNormal(seed=42)
model = Sequential()

model.add(conv_base)
model.add(Flatten())
model.add(Dense(128,activation='relu',kernel_initializer=initializer))
model.add(Dense(128,activation='relu',kernel_initializer=initializer))
model.add(Dense(2,activation='softmax',kernel_initializer=initializer))


In [7]:
model.compile(
    optimizer=tensorflow.keras.optimizers.SGD(learning_rate=0.001,momentum=0.9,nesterov=True,weight_decay=0.0005),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
  )

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 128)               3211392   
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dense_2 (Dense)             (None, 2)                 258       
                                                                 
Total params: 17942850 (68.45 MB)
Trainable params: 17942850 (68.45 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [10]:
import numpy as np
X=np.random.normal(size=(10,224,224,3))
y=np.array([1,0,1,1,0,0,1,1,0,0])

In [13]:
history=model.fit(
    X, y,
    epochs=10,
    batch_size=2
)


print(history.history['accuracy'])

Epoch 1/10
5/5 [==============================] - 27s 5s/step - loss: 1.1116 - accuracy: 0.5000
Epoch 2/10
5/5 [==============================] - 21s 4s/step - loss: 0.8070 - accuracy: 0.5000
Epoch 3/10
5/5 [==============================] - 22s 4s/step - loss: 0.7598 - accuracy: 0.3000
Epoch 4/10
5/5 [==============================] - 21s 4s/step - loss: 0.8877 - accuracy: 0.7000
Epoch 5/10
5/5 [==============================] - 21s 4s/step - loss: 0.7157 - accuracy: 0.1000
Epoch 6/10
5/5 [==============================] - 22s 4s/step - loss: 0.6930 - accuracy: 0.5000
Epoch 7/10
5/5 [==============================] - 20s 4s/step - loss: 0.7018 - accuracy: 0.5000
Epoch 8/10
5/5 [==============================] - 22s 4s/step - loss: 0.7076 - accuracy: 0.1000
Epoch 9/10
5/5 [==============================] - 21s 4s/step - loss: 0.7167 - accuracy: 0.5000
Epoch 10/10
5/5 [==============================] - 21s 4s/step - loss: 0.7016 - accuracy: 0.5000
[0.5, 0.5, 0.30000001192092896, 0.69999

KeyError: 'val_accuracy'

In [20]:
model.save('vgg16_tuned_tf')

In [15]:
! pip install -U tf2onnx
! pip install onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.7 MB/s eta 0:00:00


In [21]:
!python -m tf2onnx.convert --saved-model vgg16_tuned_tf/ --output vgg16_tf.onnx

2024-04-25 04:59:05.194600: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-25 04:59:05.194675: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-25 04:59:05.196665: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-25 04:59:06.687900: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2

In [23]:
#now converting keras model to onnx directly

In [ ]:
model.save('vgg16_tuned_keras.h5')

In [12]:
import tensorflow as tf
import tf2onnx
import onnxruntime as rt
spec = (tf.TensorSpec(
    (None,
     224, #CONFIGURATION[ "IM_SIZE"],
     224, #CONFIGURATION["IM_SIZE"]
     3), tf.float32, name="input"),)

output_path = "vgg16_keras.onnx"

model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=13, output_path=output_path)
#opset tell about the version
output_names = [n.name for n in model_proto.graph.output]

In [13]:
output_names

['dense_2']

In [14]:
#inference


import onnxruntime as rt

output_path = "vgg16_keras.onnx"
providers = ['CPUExecutionProvider']

m = rt.InferenceSession(output_path, providers=providers)

im=np.random.normal(size=(3,224,224,3))
im=np.float32(im)
m.run(output_names, {"input": im})

[array([[0.8041426 , 0.19585745],
        [0.81946695, 0.18053304],
        [0.81742567, 0.18257433]], dtype=float32)]

In [15]:
import time

t1=time.time()
model.predict(im)
print(time.time()-t1)

t1=time.time()
m.run(output_names, {"input": im})
print(time.time()-t1)

#onnx model takes lesser time

1/1 [==============================] - 2s 2s/step
2.1928510665893555
1.3798925876617432


In [43]:
rt.get_device()

'CPU'

In [44]:
#for gpu use this
! pip install onnxruntime-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.1/192.1 MB 4.7 MB/s eta 0:00:00


In [1]:
import onnxruntime as rt
rt.get_device()
#and change provider to cudaexecutionprovider

'GPU'